<img src="https://github.com/MUDSS/lectures/raw/main/semester-2-2021-2022/week-5-bootcamp-tensorflow-keras/assets/cover.png" alt="Week 5: TensorFlow & Keras">

-----------------------------

<img src="https://www.gstatic.com/devrel-devsite/prod/v2484c9574f819dcf3d7ffae39fb3001f4498b2ece38cec22517931d550e19e7d/tensorflow/images/lockup.svg" alt="TensorFlow" width='200' align='left'>

# TensorFlow Basics

TensorFlow is a free and open-source end-to-end platform for machine learning and artificial intelligence which gives developers the ability to easily build and deploy ML-powered applications. It was developed by the Google Brain team for internal Google use in research and production, then brought to public in 2015 and updated to TensorFlow 2.0 in 2019.

TensorFlow can be used in a wide variety of programming languages, including Python, C++, Javascript, Java, and could run on various platforms like desktop, server, web front-end, and embedded systems, etc.

It supports the following:

* Multidimensional-array based numeric computation (similar to <a href="https://numpy.org/" class="external">NumPy</a>.)
* GPU and distributed processing
* Automatic differentiation
* Model construction, training, and export
* And more

In [ ]:
import tensorflow as tf

tf.__version__

When [properly configured](https://www.tensorflow.org/install/gpu), TensorFlow can use accelerator hardware like GPUs to execute operations very quickly.

In [ ]:
if tf.config.list_physical_devices('GPU'):
    print("TensorFlow **IS** using the GPU")
else:
    print("TensorFlow **IS NOT** using the GPU")

## Tensors

TensorFlow operates on multidimensional arrays or _tensors_ represented as [`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) objects. Here is a two-dimensional tensor:

The most important attributes of a `tf.Tensor` are its `shape` and `dtype`:

* `Tensor.shape`: tells you the size of the tensor along each of its axes.
* `Tensor.dtype`: tells you the type of all the elements in the tensor.

TensorFlow implements standard mathematical operations on tensors, as well as many operations specialized for machine learning.

For example:

## Variables

Normal `tf.Tensor` objects are immutable. To store model weights (or other mutable state) in TensorFlow use a [`tf.Variable`](https://www.tensorflow.org/api_docs/python/tf/Variable).

## Automatic differentiation

<a href="https://en.wikipedia.org/wiki/Gradient_descent" class="external">_Gradient descent_</a> and related algorithms are a cornerstone of modern machine learning.

To enable this, TensorFlow implements automatic differentiation (autodiff), which uses calculus to compute gradients. Typically you'll use this to calculate the gradient of a model's _error_ or _loss_ with respect to its weights.

### Computing gradients

To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the *forward* pass.  Then, during the *backward pass*, TensorFlow traverses this list of operations in reverse order to compute gradients.

Here is a simple example:

In [ ]:
x = tf.Variable(1.0)

def f(x):
    return x**2 + 2*x - 5



At `x = 1.0`, `y = f(x) = (1**2 + 2*1 - 5) = -2`.

The derivative of `y` is `y' = f'(x) = (2*x + 2) = 4`. TensorFlow can calculate this automatically:

### Gradient tapes

TensorFlow provides the `tf.GradientTape` API for automatic differentiation; that is, computing the gradient of a computation with respect to some inputs, usually `tf.Variable`s.
TensorFlow "records" relevant operations executed inside the context of a `tf.GradientTape` onto a "tape". TensorFlow then uses that tape to compute the gradients of a "recorded" computation using [reverse mode differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

This simplified example only takes the derivative with respect to a single scalar (`x`), but TensorFlow can compute the gradient with respect to any number of non-scalar tensors simultaneously.

## Eager Execution and Graphs

For all codes we executed above are under a running mode of TensorFlow called *Eager Execution*. It evaluates values instantly rather than building a computation graph for later use which is compulsory in TensorFlow 1.0. Eager Execution is exactly like how Python scripts does.

While we can use TensorFlow interactively like any Python library, TensorFlow also provides tools for:

* **Performance optimization**: to speed up training and inference.
* **Export**: so you can save your model when it's done training.

These require that you use `tf.function` to separate your pure-TensorFlow code from Python.

In [ ]:

def my_func(x):
    print('Tracing...')
    y = x + 5
    return tf.reduce_sum(y)

The first time you run the `tf.function`, although it executes in Python, it captures a complete, optimized graph representing the TensorFlow computations done within the function.

On subsequent calls TensorFlow only executes the optimized graph, skipping any non-TensorFlow steps. Below, note that `my_func` doesn't print _tracing_ since `print` is a Python function, not a TensorFlow function.

A graph may not be reusable for inputs with a different _signature_ (`shape` and `dtype`), so a new graph is generated instead:

These captured graphs provide two benefits:

* In many cases they provide a significant speedup in execution.
* You can export these graphs, using `tf.saved_model`, to run on other systems like a [server](https://www.tensorflow.org/tfx/serving/docker) or a [mobile device](https://www.tensorflow.org/lite/guide), no Python installation required.

In short, graphs are extremely useful and let your TensorFlow run fast, run in parallel, and run efficiently on multiple devices.

However, you still want to define your machine learning models (or other computations) in Python for convenience, and then automatically construct graphs when you need them. And note that the process of creating graphs has some overhead.

## Modules, layers, and models

`tf.Module` is a class for managing your `tf.Variable` objects, and the `tf.function` objects that operate on them. One of significant features of Module is that we can save and load a Module anytime from or to multiple platforms. We will see this in Keras example.

The `tf.keras.layers.Layer` and `tf.keras.Model` classes build on `tf.Module` providing additional functionality and convenience methods for building, training, and saving models.

A model is, abstractly:

+ A function that computes something on tensors (a forward pass)
+ Some variables that can be updated in response to training

## Next —— tf.keras

You may have noticed that classes `Layer` and `Model` are both provided by package `tf.keras` which is the next important role for this session. In the past, Keras is an independent package based on TF, but now it has become an indispensable part of TensorFlow 2.0 ecosystem.

Keras is built upon TensorFlow and provides us high-level encapsulation on data preprocessing, building model, training, and so on. It significantly simplifies workflow of modern data scientists and enables us quickly transform data science ideas into practice.

-----------------------------

<img src="https://www.gstatic.com/devrel-devsite/prod/v2484c9574f819dcf3d7ffae39fb3001f4498b2ece38cec22517931d550e19e7d/tensorflow/images/lockup.svg" alt="TensorFlow" width='200' align='left'>

# Keras - Basic classification: Classify MNIST images

This notebook trains a neural network model to classify images of handwritten digits. This is a fast-paced overview of a complete TensorFlow program especially making use of Keras Sequential API.

This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a user-friendly high-level API to build and train models in TensorFlow, whose Sequential API helps us build models by plugging together building blocks.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import numpy as np
import matplotlib.pyplot as plt

## Import the MNIST dataset

This guide uses the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset which contains 70,000 grayscale images in 10 categories. The images show individual handwritten digit at low resolution (28 by 28 pixels), as seen here:

<img src="https://github.com/MUDSS/lectures/raw/main/semester-2-2021-2022/week-5-bootcamp-tensorflow-keras/assets/mnist_sprites.png" alt="MNIST sprite" width="600">

MNIST is often used as the **"Hello, World"** of machine learning programs for computer vision. It is a good starting point to test and debug code.

Here, 60,000 images are used to train the network and 10,000 images to evaluate how accurately the network learned to classify images. We can access the MNIST directly from Keras datasets.

Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. The *labels* are an array of integers, ranging from 0 to 9. These correspond to the *class* of the real digit.

## Preprocess the data

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255:

In [ ]:
plt.figure()
plt.imshow(train_images[0], cmap='Greys')
plt.colorbar()
plt.grid(False)
plt.show()

Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. It's important that the *training set* and the *testing set* be preprocessed in the same way:

To verify that the data is in the correct format and that you're ready to build and train the network, let's display the first 25 images from the *training set* and display the class name below each image.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

## Set up the layers

The basic building block of a neural network is the [*layer*](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

A [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

In [ ]:
model = tf.keras.Sequential([
    
], name='my_model')



The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a two-dimensional array (of 28 by 28 pixels) to a one-dimensional array (of 28 * 28 = 784 pixels). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely connected, or fully connected, neural layers. The first `Dense` layer has 128 nodes (or neurons). The second (and last) layer returns a logits array with length of 10. Each node contains a score that indicates the current image belongs to one of the 10 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's [*compile*](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) step:

* [*Optimizer*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) —This is how the model is updated based on the data it sees and its loss function.
* [*Loss function*](https://www.tensorflow.org/api_docs/python/tf/keras/losses) —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* [*Metrics*](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

## Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the `train_images` and `train_labels` arrays.
2. The model learns to associate images and labels.
3. You ask the model to make predictions about a test set—in this example, the `test_images` array.
4. Verify that the predictions match the labels from the `test_labels` array.


### Feed the model

To start training,  call the [`model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method—so called because it "fits" the model to the training data:

In [ ]:
# set up tensorboard callback
logdir = "logs/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.91 (or 91%) on the training data.

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

It turns out that the accuracy on the test dataset is a little better than the accuracy on the training dataset. This gap between training accuracy and test accuracy represents *underfitting*. Or we could say it generalises well.

### Make predictions

With the model trained, you can use it to make predictions about some images.
Attach a softmax layer to convert the model's linear outputs—[logits](https://developers.google.com/machine-learning/glossary#logits)—to probabilities, which should be easier to interpret.

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

A prediction is an array of 10 numbers. They represent the model's "confidence" that the image corresponds to each of the 10 different digits. You can see which label has the highest confidence value:

So, the model is most confident that this image is number $7$. Examining the test label shows that this classification is correct:

Graph this to look at the full set of 10 class predictions.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(predicted_label,
                                  100*np.max(predictions_array),
                                  true_label),
                                  color=color)

def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

### Verify predictions

With the model trained, you can use it to make predictions about some images.

Let's look at a certain image, predictions, and prediction array. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percentage (out of 100) for the predicted label.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Let's plot several images with their predictions. Note that the model can be wrong even when very confident.

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Save the trained model

Call [`model.save`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#save) to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*.

Saving a fully-functional model is very useful—you can load them in [TensorFlow JS](https://www.tensorflow.org/js/guide) and then train and run them in web browsers, or convert them to run on mobile devices using [TensorFlow Lite](https://www.tensorflow.org/lite/guide)

Reload a fresh Keras model from the saved model:

We could also [save checkpoints during training](https://www.tensorflow.org/tutorials/keras/save_and_load#save_checkpoints_during_training), which allows us to pick-up training where we left off in case the training process was interrupted. The [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) callback allows us to continually save the model both *during* and at *the end* of training.

## Use the loaded model

Finally, use the loaded model to make a prediction about a single image.

`tf.keras` models are optimized to make predictions on a *batch*, or collection, of examples at once. Accordingly, even though you're using a single image, you need to add it to a list:

Now predict the correct label for this image:

`tf.keras.Model.predict` returns a list of lists—one list for each image in the batch of data. Grab the predictions for our (only) image in the batch:

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
plt.xticks(range(10), list(range(10)))
plt.show()

And the model predicts a label as expected.

In [ ]:
if np.argmax(predictions_single[0]) == test_labels[1]:
    pass

## Know TensorBoard
[TensorBoard](https://tensorflow.google.cn/tensorboard) is a tool for providing the measurements and visualizations needed during the machine learning workflow. It enables tracking experiment metrics like loss and accuracy, visualizing the model graph, projecting embeddings to a lower dimensional space, and much more.

Recall that previously we have added a callback method into the training process. This logs all necessary information needed by TensorBoard and we could easily visualise what we did.

In [ ]:
# Open an embedded TensorBoard viewer
%tensorboard --logdir {logdir}

## WebApp

We are going to build a web application based on [*TensorFlow.js*](https://www.tensorflow.org/js/guide) and use our trained model. It accepts real user drawing of digit and makes prediction on that.

To Convert a saved model to a web format model usable by tf.js, do the following:

```
% pip install tensorflowjs
% tensorflowjs_converter --input_format=tf_saved_model saved_model/ web_model/
```

<img src="./assets/webapp.png" alt="webApp">